In [ ]:
!pip install transformers
!pip install datasets
!pip install apache_beam
!pip install sentencepiece

In [1]:
import random
import re

import gc

random.seed(0)
print(random.random()) 

0.8444218515250481


In [2]:
from datasets import load_dataset

dataset = load_dataset("oscar", "unshuffled_deduplicated_sa")

Dataset oscar downloaded and prepared to C:\Users\surajp\.cache\huggingface\datasets\oscar\unshuffled_deduplicated_sa\1.0.0\84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 90.93it/s]


In [ ]:
text = dataset["train"]["text"]

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = tfds.load('wikipedia/20200301.sa', split="train")

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/3 [00:00<?, ? file/s]


Dataset wikipedia downloaded and prepared to /root/tensorflow_datasets/wikipedia/20200301.sa/1.0.0. Subsequent calls will reuse this data.


In [ ]:
df = tfds.as_dataframe(ds.take(len(ds)))
txt = df.text.apply(lambda x: x.decode('utf-8'))
gc.collect()

txt_ = dataset["train"]["text"]

text = list(txt_)+list(txt)
text = [re.sub("[a-zA-Z0-9]+", "", txt_) for txt_ in text]
gc.collect()

del txt
del txt_
gc.collect()

0

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name, strip_accents=False)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForMaskedLM: ['sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config

AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "vocab_size": 200000
}

In [ ]:
tokenizer.init_kwargs

{'bos_token': '[CLS]',
 'cls_token': '[CLS]',
 'do_lower_case': True,
 'eos_token': '[SEP]',
 'keep_accents': False,
 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False),
 'name_or_path': 'ai4bharat/indic-bert',
 'pad_token': '<pad>',
 'remove_space': True,
 'sep_token': '[SEP]',
 'sp_model_kwargs': {},
 'special_tokens_map_file': None,
 'strip_accents': False,
 'tokenizer_file': None,
 'unk_token': '<unk>'}

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def batch_encode(text, max_seq_len, batch_size=4096):
    encoded_sentences = []
    for i in tqdm(range(0, len(text), batch_size)):
        encoded_sent = tokenizer.batch_encode_plus(
            text[i : i + batch_size],
            max_length=max_seq_len,
            add_special_tokens=True,
            padding=True,
            return_attention_mask=True,
            pad_to_max_length=True,
            truncation=True,
            #return_tensors="pt",
        )

    encoded_sentences += encoded_sent
    
    return (input_ids_train, attention_masks_train)

In [ ]:
#encoded_sentences = batch_encode(text=text,
#                                                      max_seq_len=128, 
#                                                      batch_size=4096)

In [ ]:
import torch

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
X_train_tokenized = tokenizer(text, padding=True, truncation=True, max_length=128)

In [ ]:
train_dataset = Dataset(X_train_tokenized)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
#https://huggingface.co/docs/transformers/v4.17.0/en/main_classes/trainer#transformers.TrainingArguments

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=40,
    logging_strategy ="epoch",
    weight_decay=0.01,
    do_train=True,
    save_strategy ="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 29090
  Num Epochs = 40
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 72760


Step,Training Loss
1819,2.754100


Saving model checkpoint to ./results/checkpoint-1819
Configuration saved in ./results/checkpoint-1819/config.json
Model weights saved in ./results/checkpoint-1819/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1819/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1819/special_tokens_map.json


In [ ]:
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name, strip_accents=False)
model = AutoModelForMaskedLM.from_pretrained(model_name)